In [40]:
from src.fforma import FFORMA
from sklearn.decomposition import PCA
from src.meta_learner import MetaLearnerNN
from src.meta_results_r_data import prepare_fforma_data

from ESRNN.m4_data import prepare_m4_data, seas_dict
from ESRNN.utils_evaluation import evaluate_prediction_owa

from src.utils import (
    evaluate_forecasts,
    freqs
)
from src.benchmarks import FactorQuantileRegressionAveraging

import pandas as pd
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pickle

from statsmodels.regression.quantile_regression import QuantReg

from src.metrics.metrics import smape, mape
from src.benchmarks import evaluate_panel

In [2]:
data = prepare_fforma_data(directory='data', dataset_name=None, kind='TOURISM')
#dataset_name=None for all data

% of missing values, train set
x_acf10               7.551487
diff1_acf1            7.551487
diff1_acf10           7.551487
diff2_acf1            7.551487
diff2_acf10           8.543097
seas_acf1            39.511823
arch_lm               9.687262
garch_acf             0.076278
arch_r2              13.196034
garch_r2             13.348589
nonlinearity          1.144165
x_pacf5               1.144165
diff1x_pacf5          1.754386
diff2x_pacf5          2.059497
seas_pacf            39.511823
e_acf10               7.551487
seasonal_strength    39.511823
peak                 39.511823
trough               39.511823
hw_alpha             39.511823
hw_beta              39.511823
hw_gamma             39.511823
dtype: float64


% of missing values, test set
x_acf10               1.754386
diff1_acf1            1.754386
diff1_acf10           2.059497
diff2_acf1            2.059497
diff2_acf10           2.212052
seas_acf1            39.511823
arch_lm               7.551487
arch_r2               7

In [3]:
X_train = data['preds_train_df']
y_train = data['y_train_df']
X_test = data['preds_test_df']
y_test = data['y_test_df']

In [39]:
unique_ids = X_train['unique_id'].unique()
smape_list = []
y_hat_list = []
u_id_list = []
ds_list = []
for u_id in unique_ids:
    x_id = X_train[X_train['unique_id']==u_id].drop(columns=['unique_id','ds']).values
    x_test = X_test[X_test['unique_id']==u_id].drop(columns=['unique_id','ds']).values
    y_id = y_train[y_train['unique_id']==u_id]['y'].values
    y_test_id = y_test[y_test['unique_id']==u_id]['y'].values

    pca_model = PCA(n_components=1).fit(x_id)
    X = pca_model.transform(x_id)
    X = np.hstack([X, np.ones((len(X),1))])

    reg = QuantReg(y_id, X).fit(0.5)

    x_test_pca = pca_model.transform(x_test)
    x_test_pca = np.hstack([x_test_pca, np.ones((len(x_test_pca),1))])
    y_hat = reg.predict(x_test_pca)

    my_smape=200*np.mean(np.abs(y_hat-y_test_id)/(np.abs(y_hat)+np.abs(y_test_id)))
    smape_list.append(my_smape)
    y_hat_list += y_hat.tolist()
    u_id_list += [u_id]*len(y_hat)
    ds_list += list(range(len(y_hat)))
    
df_hat = pd.DataFrame(list(zip(u_id_list, ds_list, y_hat_list)), 
               columns =['unique_id', 'ds', 'y_hat']) 
    
np.mean(smape_list)

33.242674523229795

In [44]:
evaluate_panel(y_test, df_hat, smape)

[########################################] | 100% Completed |  0.3s


34.83226943604919

In [5]:
fqra = FactorQuantileRegressionAveraging(n_components=1, tau=0.5)

In [6]:
fqra.fit(X_train, y_train, X_test, y_test)

[########################################] | 100% Completed | 50.5s
[########################################] | 100% Completed | 25.1s
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.5s


In [9]:
fqra.test_min_smape

33.4804924701254